# Imports and Plotting functions
Collapse the next cell if you wish

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import ipywidgets as widgets

def get_data_all(deception_informed, beta, measure, suffix=''):
    # read pickle file
    pred0_df = pd.read_pickle('results/{}_prediction_0_beta{}/data{}.pkl'.format(deception_informed, beta, suffix)).rename(columns={'{}_mean'.format(measure): 'prediction_0_{}_mean'.format(measure), '{}_std'.format(measure): 'prediction_0_{}_std'.format(measure)})[['step', 'prediction_0_{}_mean'.format(measure), 'prediction_0_{}_std'.format(measure)]]
    pred1_df = pd.read_pickle('results/{}_prediction_1_beta{}/data{}.pkl'.format(deception_informed, beta, suffix)).rename(columns={'{}_mean'.format(measure): 'prediction_1_{}_mean'.format(measure), '{}_std'.format(measure): 'prediction_1_{}_std'.format(measure)})[['prediction_1_{}_mean'.format(measure), 'prediction_1_{}_std'.format(measure)]]
    ambiguity_df = pd.read_pickle('results/{}_ambiguity_beta{}/data{}.pkl'.format(deception_informed, beta, suffix)).rename(columns={'{}_mean'.format(measure): 'ambiguity_{}_mean'.format(measure), '{}_std'.format(measure): 'ambiguity_{}_std'.format(measure)})[['ambiguity_{}_mean'.format(measure), 'ambiguity_{}_std'.format(measure)]]
    exaggeration_df = pd.read_pickle('results/{}_exaggeration_beta{}/data{}.pkl'.format(deception_informed, beta, suffix)).rename(columns={'{}_mean'.format(measure): 'exaggeration_{}_mean'.format(measure), '{}_std'.format(measure): 'exaggeration_{}_std'.format(measure)})[['exaggeration_{}_mean'.format(measure), 'exaggeration_{}_std'.format(measure)]]
    to_plot = pd.concat([pred0_df, pred1_df, ambiguity_df, exaggeration_df], axis=1)

    return to_plot

def pooled_std(stds, sizes):
    numerator = sum((n - 1) * s ** 2 for s, n in zip(stds, sizes))
    denominator = sum(sizes) - len(sizes)
    if denominator <= 0:
        raise ValueError("Invalid sample sizes; the denominator becomes non-positive.")
    return math.sqrt(numerator / denominator)

def plot_bars_on_ax(ax, to_plot, flag, beta, measure, manipulated=['prediction_0', 'prediction_1', 'ambiguity', 'exaggeration'], total_num_participants=316):
    # Get the current Seaborn color palette (default is 'deep')
    colors = sns.color_palette("deep", 4)

    # Width of a bar 
    width = 0.2       

    # Position of bars on x axis
    r1 = np.arange(len(to_plot))
    r2 = [x + width for x in r1]
    r3 = [x + width for x in r2]
    r4 = [x + width for x in r3]
    r_list = [r1, r2, r3, r4]

    # Creating bars
    if ('prediction_0' in manipulated):
        ind = manipulated.index('prediction_0')
        r1_mean = "{:.3f}".format(to_plot['prediction_0_{}_mean'.format(measure)].mean())
        r1_pooled_std = "{:.3f}".format(pooled_std(to_plot['prediction_0_{}_std'.format(measure)], [total_num_participants]*len(to_plot)))
        ax.bar(r_list[ind], to_plot['prediction_0_{}_mean'.format(measure)], width=width, yerr=to_plot['prediction_0_{}_std'.format(measure)], capsize=4, label='prediction 0 (g2+g3): (mm:{}, p_std:{})'.format(r1_mean, r1_pooled_std), color=colors[ind])
    if ('prediction_1' in manipulated):
        ind = manipulated.index('prediction_1')
        r2_mean = "{:.3f}".format(to_plot['prediction_1_{}_mean'.format(measure)].mean())
        r2_pooled_std = "{:.3f}".format(pooled_std(to_plot['prediction_1_{}_std'.format(measure)], [total_num_participants]*len(to_plot)))
        ax.bar(r_list[ind], to_plot['prediction_1_{}_mean'.format(measure)], width=width, yerr=to_plot['prediction_1_{}_std'.format(measure)], capsize=4, label='prediction 1 (g1+g2): (mm:{}, p_std:{})'.format(r2_mean, r2_pooled_std), color=colors[ind])
    if ('ambiguity' in manipulated):
        ind = manipulated.index('ambiguity')
        r3_mean = "{:.3f}".format(to_plot['ambiguity_{}_mean'.format(measure)].mean())
        r3_pooled_std = "{:.3f}".format(pooled_std(to_plot['ambiguity_{}_std'.format(measure)], [total_num_participants]*len(to_plot)))
        ax.bar(r_list[ind], to_plot['ambiguity_{}_mean'.format(measure)], width=width, yerr=to_plot['ambiguity_{}_std'.format(measure)], capsize=4, label='ambiguity: (mm:{}, p_std:{})'.format(r3_mean, r3_pooled_std), color=colors[ind])
    if ('exaggeration' in manipulated):
        ind = manipulated.index('exaggeration')
        r4_mean = "{:.3f}".format(to_plot['exaggeration_{}_mean'.format(measure)].mean())
        r4_pooled_std = "{:.3f}".format(pooled_std(to_plot['exaggeration_{}_std'.format(measure)], [total_num_participants]*len(to_plot)))
        ax.bar(r_list[ind], to_plot['exaggeration_{}_mean'.format(measure)], width=width, yerr=to_plot['exaggeration_{}_std'.format(measure)], capsize=4, label='exaggeration: (mm:{}, p_std:{})'.format(r4_mean, r4_pooled_std), color=colors[ind])

    # Adding xticks
    ax.set_xticks([r + width for r in range(len(to_plot))])
    ax.set_xticklabels(to_plot['step'])

    ax.set_title('{}_{}_{}'.format(flag, beta, measure))
    ax.legend(loc='upper left', bbox_to_anchor=(1, 1))

def plot_lines_on_ax(ax, to_plot, flag, beta, measure, manipulated=['prediction_0', 'prediction_1', 'ambiguity', 'exaggeration'], total_num_participants=316):
    # Get the current Seaborn color palette (default is 'deep')
    colors = sns.color_palette("deep", 4)

    # Creating bars
    if ('prediction_0' in manipulated):
        ind = manipulated.index('prediction_0')
        r1_mean = "{:.3f}".format(to_plot['prediction_0_{}_mean'.format(measure)].mean())
        r1_pooled_std = "{:.3f}".format(pooled_std(to_plot['prediction_0_{}_std'.format(measure)], [total_num_participants]*len(to_plot)))
        ax.plot(to_plot['step'], to_plot['prediction_0_{}_mean'.format(measure)], marker = 'o', label='prediction 0 (g2+g3): (mm:{}, p_std:{})'.format(r1_mean, r1_pooled_std), color=colors[ind])
        plt.fill_between(to_plot['step'], to_plot['prediction_0_{}_mean'.format(measure)] - to_plot['prediction_0_{}_std'.format(measure)], to_plot['prediction_0_{}_mean'.format(measure)] + to_plot['prediction_0_{}_std'.format(measure)], color=colors[ind], alpha=0.15)
    if ('prediction_1' in manipulated):
        ind = manipulated.index('prediction_1')
        r2_mean = "{:.3f}".format(to_plot['prediction_1_{}_mean'.format(measure)].mean())
        r2_pooled_std = "{:.3f}".format(pooled_std(to_plot['prediction_1_{}_std'.format(measure)], [total_num_participants]*len(to_plot)))
        ax.plot(to_plot['step'], to_plot['prediction_1_{}_mean'.format(measure)], marker = 's', label='prediction 1 (g1+g2): (mm:{}, p_std:{})'.format(r2_mean, r2_pooled_std), color=colors[ind])
        plt.fill_between(to_plot['step'], to_plot['prediction_1_{}_mean'.format(measure)] - to_plot['prediction_1_{}_std'.format(measure)], to_plot['prediction_1_{}_mean'.format(measure)] + to_plot['prediction_1_{}_std'.format(measure)], color=colors[ind], alpha=0.15)
    if ('ambiguity' in manipulated):
        ind = manipulated.index('ambiguity')
        r3_mean = "{:.3f}".format(to_plot['ambiguity_{}_mean'.format(measure)].mean())
        r3_pooled_std = "{:.3f}".format(pooled_std(to_plot['ambiguity_{}_std'.format(measure)], [total_num_participants]*len(to_plot)))
        ax.plot(to_plot['step'], to_plot['ambiguity_{}_mean'.format(measure)], marker = '^', label='ambiguity: (mm:{}, p_std:{})'.format(r3_mean, r3_pooled_std), color=colors[ind])
        plt.fill_between(to_plot['step'], to_plot['ambiguity_{}_mean'.format(measure)] - to_plot['ambiguity_{}_std'.format(measure)], to_plot['ambiguity_{}_mean'.format(measure)] + to_plot['ambiguity_{}_std'.format(measure)], color=colors[ind], alpha=0.15)
    if ('exaggeration' in manipulated):
        ind = manipulated.index('exaggeration')
        r4_mean = "{:.3f}".format(to_plot['exaggeration_{}_mean'.format(measure)].mean())
        r4_pooled_std = "{:.3f}".format(pooled_std(to_plot['exaggeration_{}_std'.format(measure)], [total_num_participants]*len(to_plot)))
        ax.plot(to_plot['step'], to_plot['exaggeration_{}_mean'.format(measure)], marker = '*', label='exaggeration: (mm:{}, p_std:{})'.format(r4_mean, r4_pooled_std), color=colors[ind])
        plt.fill_between(to_plot['step'], to_plot['exaggeration_{}_mean'.format(measure)] - to_plot['exaggeration_{}_std'.format(measure)], to_plot['exaggeration_{}_mean'.format(measure)] + to_plot['exaggeration_{}_std'.format(measure)], color=colors[ind], alpha=0.15)

    ax.set_title('{}_{}_{}'.format(flag, beta, measure))
    ax.legend(loc='upper left', bbox_to_anchor=(1, 1))

def plot_data(flag, beta, measure, manipulated, plot_type='lines'):
    fig, ax = plt.subplots(figsize=(10, 4))
    if plot_type == 'bars':
        plot_bars_on_ax(ax, get_data_all(flag, beta, measure), flag, beta, measure, manipulated)
    elif plot_type == 'lines':
        plot_lines_on_ax(ax, get_data_all(flag, beta, measure), flag, beta, measure, manipulated)
    plt.tight_layout()
    plt.show()

# Interactive Plots
Collapse the next cell if you wish

In [17]:
# Define the dropdown options for each category
flag_options = ['YES', 'NO']
beta_options = ['1.5', '6']
measure_options = ['scores', 'correctness', 'confidence', 'goal_selection', 'confidence_reaction']

# Create the dropdown widgets
flag_dropdown = widgets.Dropdown(options=flag_options, description='Group:')
beta_dropdown = widgets.Dropdown(options=beta_options, description='Beta:')
measure_dropdown = widgets.Dropdown(options=measure_options, description='Measure:')
policy_type_checkbox = widgets.SelectMultiple(options=['prediction_0', 'prediction_1', 'ambiguity', 'exaggeration'], value=['prediction_0', 'prediction_1', 'ambiguity', 'exaggeration'], description='Policy types:')
plot_type_radio = widgets.RadioButtons(options=['bars', 'lines'], value='lines', description='Plot type:')

print('To multi-select policy types, hold down the Ctrl key on Windows or the Command key on Mac.')
widgets.interactive(plot_data, flag=flag_dropdown, beta=beta_dropdown, measure=measure_dropdown, manipulated=policy_type_checkbox, plot_type=plot_type_radio)

To multi-select policy types, hold down the Ctrl key on Windows or the Command key on Mac.


interactive(children=(Dropdown(description='Group:', options=('YES', 'NO'), value='YES'), Dropdown(description…